In [ ]:
pip install pandas 

In [6]:
import pandas as pd #import pandas

In [11]:
df = pd.read_csv("data.csv" , low_memory=False) #read the dataset in a dataframe to utilize the data in python.

In [ ]:
pip install TextBlob

In [13]:
from textblob import TextBlob #import TextBlob to procces the text.

In [40]:
#this function will procces the reviews to get the sentiment from it.
def polarity(review):
    score = TextBlob(review)
    return score.sentiment.polarity

In [29]:
df["sentiment_score"] = df["reviews.text"].astype(str).apply(sentiment)

In [45]:
def classifyer(score): #classify each review based on the sentiment score as good/bad/Neutral.
    if score > 0.2:
        return 'Good'
    elif score < -0.2:
        return 'Bad'
    elif score >= -0.2 and score <= 0.2:
        return 'Neutral'
    

In [46]:
df["sentimentAnalysis"] = df["sentiment_score"].apply(classifyer) #call and apply the classifyer function on a new column named sentimentAnalysis.

In [47]:
df[["sentiment_score","sentimentAnalysis"]].tail(25)

,sentiment_score,sentimentAnalysis
34635,0.059896,Neutral
34636,0.275238,Good
34637,0.228125,Good
34638,0.292133,Good
34639,0.035156,Neutral
34640,0.136364,Neutral
34641,0.135185,Neutral
34642,0.225000,Good
34643,0.643750,Good
34644,0.250000,Good


In [49]:
#A description of the project.?
"""Answer:Getting sentiment analysis of products reviews from a dataset and making sense of the data based on polarity score of each one to classfy them as good,bad or neutral. """
#A description of the dataset used.?
"""Answer:Dataset of products and thier reviews."""
#A description of the methods used to analyze the data.?
"""Answer:A polarity function that process the reviews to get the sentiment from it.
 then use the Classifyer function to classify each review based on the sentiment score as good/bad/Neutral."""
#A description of the results.?
"""Answer: the result is based on the classifyer function which will display good if sentiment score over 0.2, bad if score under 0.2 and neutral in between."""

'Answer: the result is based on the classifyer function which will display good if sentiment score over 0.2, bad if score under 0.2 and neutral in between.'